## **Mount Drive**

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/VIT/Tamil Argumentation

/content/gdrive/MyDrive/VIT/Tamil Argumentation


## **Install**

In [3]:
pip install transformers nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.2 MB/s eta 0:00:00


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## **Import Libraries**

In [5]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from copy import deepcopy

from sklearn import metrics
from sklearn.model_selection import KFold

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer,AutoModel,BertTokenizer

from nltk.tokenize import word_tokenize

## **Import Dataset**

In [6]:
df = pd.read_excel("/content/gdrive/MyDrive/VIT/Tamil Argumentation/Twitter Comment Dataset.xlsx")

In [7]:
df.head()

,S No,Tweet,Date of Tweet,Topic,Parent Tweet,Language,Quality,Stance,Argument,Comment,Responding to Tone,Discussing Writer Characteristics,Remark,Relevancy
0,1,"Bro imagine today is Friday , big star movie i...",2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",ENGLISH,Med,Undetermined,0,1,0,0,0,Relevant
1,2,Dei unnoda akkarai TN mela not on others and w...,2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",ENGLISH,Med,Against,0,1,0,1,0,Relevant
2,3,En ninga ivara matum mention panuringa naraiya...,2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",CODE-MIXED,Med,For,0,1,0,0,0,Relevant
3,4,What is happening in Thoothukudi is totally no...,2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",ENGLISH,High,Against,1,1,0,0,0,Relevant
4,5,Ungaluku Sterlite protest prachanaya illa Bala...,2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",CODE-MIXED,Med,Undetermined,0,0,1,0,0,Relevant


In [8]:
df.columns

Index(['S No', 'Tweet', 'Date of Tweet', 'Topic', 'Parent Tweet', 'Language',
       'Quality', 'Stance', 'Argument', 'Comment', 'Responding to Tone',
       'Discussing Writer Characteristics', 'Remark', 'Relevancy'],
      dtype='object')

## **Load Text and Labels**

In [9]:
text = df["Tweet"].to_numpy()
pt = df["Parent Tweet"].to_numpy()
topic = df["Topic"].to_numpy()

Language_label = df["Language"].to_numpy()
Stance_label = df["Stance"].to_numpy()
Quality_label = df["Quality"].to_numpy()
Argument_label = df["Argument"].to_numpy()
Comment_label = df["Comment"].to_numpy()
Writer_label = df["Discussing Writer Characteristics"].to_numpy()
Tone_label = df["Responding to Tone"].to_numpy()
Remark_label = df["Remark"].to_numpy()
Relevancy_label = df["Relevancy"].to_numpy()

## **Label Encoding**

In [10]:
encode_dict_quality = {
    "High": np.array([1, 0, 0]),
    "Med": np.array([0, 1, 0]),
    "Low": np.array([0, 0, 1]),
}

encode_dict_language = {
    "ENGLISH": np.array([1, 0, 0]),
    "TAMIL": np.array([0, 1, 0]),
    "CODE-MIXED": np.array([0, 0, 1]),
}

encode_dict_stance = {
    "For": np.array([1, 0, 0]),
    "Against": np.array([0, 1, 0]),
    "Undetermined": np.array([0, 0, 1]),
}

encode_dict = {
    0: np.array([1, 0]),
    1: np.array([0, 1]),
}

encode_dict_relevancy = {
    "Relevant": np.array([1, 0]),
    "Irrelevant": np.array([0, 1]),
}

In [11]:
Language_label = np.array([encode_dict_language[label] for label in Language_label])
Quality_label = np.array([encode_dict_quality[label] for label in Quality_label])
Argument_label = np.array([encode_dict[label] for label in Argument_label])
Comment_label = np.array([encode_dict[label] for label in Comment_label])
Writer_label = np.array([encode_dict[label] for label in Writer_label])
Tone_label = np.array([encode_dict[label] for label in Tone_label])
Remark_label = np.array([encode_dict[label] for label in Remark_label])
Relevancy_label = np.array([encode_dict_relevancy[label] for label in Relevancy_label])
Stance_label = np.array([encode_dict_stance[label] for label in Stance_label])

## **Pre-Config for mBERT**

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

MAX_LEN = np.max([(len(x)+len(y)+1) for x,y in zip(text,pt)])
MAX_LEN = np.min([MAX_LEN, 510])

BATCH_SIZE = 32
LEARNING_RATE = 1e-1

In [13]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')

## **Build Dataset for mBERT**

In [14]:
class ModelDataset(Dataset):
    def __init__(self, X, PT, topic, y, tokenizer, max_len):
        self.max_len = max_len
        self.text = X
        self.topic = topic
        self.pt = PT
        self.tokenizer = tokenizer
        self.targets = y

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        text = self.text[index]
        pt = self.pt[index]
        topic = self.topic[index]

        concat_text = f"{topic}. {pt}"

        inputs = self.tokenizer(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True)

        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        return {
            'ids': torch.tensor(ids, dtype=torch.long).to(device),
            'mask': torch.tensor(mask, dtype=torch.long).to(device),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long).to(device),
            'targets': torch.tensor(self.targets[index], dtype=torch.float).to(device),
        }

## **Build Model**

In [24]:
class CustomModel(nn.Module):

    def __init__(self):
        super(CustomModel, self).__init__()
        self.bert = AutoModel.from_pretrained('xlm-roberta-large')

        for param in self.bert.parameters():
            param.requires_grad = False

        self.out_layer = nn.Linear(1024, 3)

    def forward(self, ids, mask, token_type_ids):
        _, features = self.bert(
            ids, token_type_ids=token_type_ids,
            attention_mask=mask, return_dict=False
        )

        output = self.out_layer(features)

        return output

## **Train Model**

In [16]:
def train(epoch, model, train_loader, loss_fn, optimizer):

    model.train()

    for batch in tqdm(train_loader):

        optimizer.zero_grad()

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()


    print(f'Epoch: {epoch + 1}, Loss:  {loss.item()}')

In [17]:
def validation(data_loader, model):

    model.eval()
    targets = []
    outputs = []

    with torch.no_grad():

        for batch in data_loader:

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
            batch_targets = batch['targets'].to(device, dtype = torch.float)

            batch_outputs = model(ids, mask, token_type_ids)

            targets.extend(batch_targets.cpu().numpy().tolist())
            outputs.extend(batch_outputs.cpu().numpy().tolist())

    return outputs, targets

# Model initialization

In [25]:
kf = KFold(n_splits=5)

model_targets = []
model_labels = []

model = CustomModel().to(device)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
ksi = 0

for train_index, test_index in kf.split(text):

    text_train, text_test = text[train_index], text[test_index]
    pt_train, pt_test = pt[train_index], pt[test_index]
    topic_train, topic_test = topic[train_index], topic[test_index]
    labels_train, labels_test = Quality_label[train_index], Quality_label[test_index]

    train_data = ModelDataset(text_train, pt_train, topic_train, labels_train, tokenizer, MAX_LEN)
    test_data = ModelDataset(text_test, pt_test, topic_test, labels_test, tokenizer, MAX_LEN)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

    best_score = -np.inf
    best_weights = None

    EPOCHS = 5

    print(f"Split   ---->    {ksi}")
    ksi+=1

    torch.save(model.state_dict(),"/content/gdrive/MyDrive/VIT/Tamil Argumentation/twitter_model_argument_w3i.pth")

    for epoch in range(EPOCHS):

        train(epoch, model, train_loader, loss_fn, optimizer)
        outputs, targets = validation(test_loader, model)

        print(outputs)
        print(" ")
        print(targets)

        targets = [np.argmax(x) for x in targets]
        outputs = [np.argmax(x) for x in outputs]

        score = metrics.f1_score(targets, outputs, average='weighted')

        if score > best_score:
            best_score = score
            best_weights = deepcopy(model.state_dict())

    model.load_state_dict(best_weights)
    torch.save(model.state_dict(),"/content/gdrive/MyDrive/VIT/Tamil Argumentation/twitter_model_argument_w3i.pth")

    outputs, targets = validation(test_loader, model)

    targets = [np.argmax(x) for x in targets]
    outputs = [np.argmax(x) for x in outputs]

    model_targets.extend(targets)
    model_labels.extend(outputs)

Split   ---->    0


100%|██████████| 34/34 [01:48<00:00,  3.18s/it]


Epoch: 1, Loss:  1.86967134475708
[[2.1826131343841553, 0.8460273742675781, -8.266489028930664], [0.13294443488121033, 1.4864377975463867, -7.731884479522705], [2.0893473625183105, 0.9235100746154785, -8.412938117980957], [3.6496565341949463, 0.7178024053573608, -9.590620040893555], [1.067246913909912, 1.8587608337402344, -9.010542869567871], [0.32825735211372375, 1.7714769840240479, -8.11626148223877], [0.9991947412490845, 1.4064373970031738, -8.100919723510742], [1.6864638328552246, 1.1849333047866821, -8.275257110595703], [1.6346983909606934, 0.6264225244522095, -7.35408878326416], [1.8790805339813232, 1.2136375904083252, -8.720536231994629], [1.8249820470809937, 0.8372930288314819, -8.647817611694336], [0.1165735051035881, 2.0016086101531982, -8.654379844665527], [0.9342275261878967, 1.3152744770050049, -8.043020248413086], [3.0170130729675293, 1.321244239807129, -10.134750366210938], [1.5941863059997559, 1.1292259693145752, -8.106264114379883], [2.751443386077881, 0.78596895933151

100%|██████████| 34/34 [01:58<00:00,  3.49s/it]


Epoch: 2, Loss:  0.9774911999702454
[[-10.12330150604248, 2.6109023094177246, -3.8482415676116943], [-13.05786418914795, 3.3790130615234375, -2.7519423961639404], [-11.703072547912598, 2.340658187866211, -2.541114330291748], [-9.189213752746582, 2.196193218231201, -4.494356155395508], [-12.215758323669434, 3.615398406982422, -3.9285056591033936], [-12.594670295715332, 3.832651138305664, -3.5448226928710938], [-11.546026229858398, 3.3478569984436035, -3.7291030883789062], [-10.902073860168457, 3.006666421890259, -3.714461326599121], [-10.471332550048828, 2.178392171859741, -2.9657657146453857], [-12.431539535522461, 2.761444330215454, -2.5574464797973633], [-12.501795768737793, 2.2991878986358643, -2.4404900074005127], [-13.566253662109375, 3.9745919704437256, -3.5654549598693848], [-12.7380952835083, 2.8395519256591797, -2.4718334674835205], [-10.40245532989502, 2.9799349308013916, -4.858643054962158], [-12.19206714630127, 2.625493049621582, -2.328030824661255], [-11.446743965148926, 2

...

100%|██████████| 34/34 [01:58<00:00,  3.48s/it]


Epoch: 2, Loss:  0.7322126030921936
[[-2.472571611404419, 2.438709020614624, -1.7440590858459473], [-1.81304132938385, 1.9176322221755981, -1.7160751819610596], [-4.8065643310546875, 2.2065436840057373, 0.5003914833068848], [-3.298182487487793, 2.2643535137176514, -0.5644938945770264], [-3.6649789810180664, 2.8440792560577393, -1.14921236038208], [-3.035533905029297, 2.527048349380493, -0.6171132326126099], [-3.6125805377960205, 2.8414554595947266, -0.9565404653549194], [-7.21363639831543, 2.785344362258911, -0.09914661943912506], [-2.663844108581543, 1.9994938373565674, -0.4495460093021393], [-3.2638192176818848, 2.540938377380371, -1.0805696249008179], [-3.1836109161376953, 2.046156644821167, -0.9866297245025635], [-2.862037181854248, 1.774968147277832, 0.1738433539867401], [-2.4487171173095703, 2.243077516555786, -1.0111916065216064], [-4.345184326171875, 2.6464550495147705, -0.33195754885673523], [-0.6555485725402832, 1.4257453680038452, -2.4279398918151855], [-2.0466737747192383, 

100%|██████████| 34/34 [01:59<00:00,  3.51s/it]


Epoch: 3, Loss:  0.4751746356487274
[[-2.2149369716644287, -0.19929850101470947, -3.186758279800415], [-1.4653003215789795, -0.6505374312400818, -3.1619787216186523], [-5.539823055267334, -0.4903339147567749, -0.6802293658256531], [-3.459120988845825, -0.4579707384109497, -1.8711681365966797], [-4.049154758453369, 0.2278342843055725, -2.455399513244629], [-3.4993512630462646, -0.14836758375167847, -1.8590598106384277], [-4.110940456390381, 0.18049633502960205, -2.1619210243225098], [-6.2530436515808105, 0.09437548369169235, -2.070899724960327], [-3.081195116043091, -0.5979599952697754, -1.737074613571167], [-3.34696364402771, -0.13983523845672607, -2.4008545875549316], [-2.852018117904663, -0.5089685916900635, -2.5819296836853027], [-3.287689685821533, -0.9403166770935059, -1.0737439393997192], [-2.4359443187713623, -0.42569541931152344, -2.3406660556793213], [-4.990674018859863, -0.06949426978826523, -1.567212462425232], [0.3838166296482086, -1.0888690948486328, -4.051980495452881], [

  0%|          | 0/34 [00:00<?, ?it/s]

In [20]:
accuracy = metrics.accuracy_score(model_targets, model_labels)
f1_score_w_avg = metrics.f1_score(model_targets, model_labels, average='weighted')

print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Weighted) = {f1_score_w_avg}")

print(metrics.classification_report(model_targets, model_labels))

Accuracy Score = 0.8511111111111112
F1 Score (Weighted) = 0.8006801971887129
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      1139
           1       0.67      0.09      0.17       211

    accuracy                           0.85      1350
   macro avg       0.76      0.54      0.54      1350
weighted avg       0.83      0.85      0.80      1350



# Tester

In [23]:
test_data = ModelDataset(text, pt, topic, Argument_label, tokenizer, MAX_LEN)

test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

outputs, targets = validation(test_loader, model)

targets = [np.argmax(x) for x in targets]
outputs = [np.argmax(x) for x in outputs]

accuracy = metrics.accuracy_score(targets, outputs)
f1_score_w_avg = metrics.f1_score(targets, outputs, average='weighted')

print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Weighted) = {f1_score_w_avg}")

print(metrics.classification_report(targets, outputs))

Accuracy Score = 0.8466666666666667
F1 Score (Weighted) = 0.7818742258516321
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      1139
           1       0.75      0.03      0.05       211

    accuracy                           0.85      1350
   macro avg       0.80      0.51      0.49      1350
weighted avg       0.83      0.85      0.78      1350



# Tokenization testing

In [ ]:
print(tokenizer.decode([ 101, 28248, 35732, 22044,   118, 21631, 10345, 11101, 16602,   119,
        10159, 48502, 10107, 10108,   146, 16994, 10806,   117, 10231, 87150,
        22525, 22489, 11309, 10161, 21528, 10114, 63376, 11915, 10135,   108,
        10201, 35732, 22044,   123, 31081, 10169, 22528, 10114, 52824, 10123,
        11345,   119, 12882, 14796, 10944,   189, 11419, 51511, 51354, 10188,
        11049, 11309, 10161,   136,   102, 16938,   112,   188, 45476, 10114,
        23763, 10531,   119,   119,   119, 10678, 10114, 94992, 10219, 10111,
        23763, 10105, 11561, 40414,   119, 14453, 44096,   189, 11337, 10678,
        10114, 21852, 10479, 10301, 22489, 11426,   102]))

[CLS] Jalikattu - Ban or Allow. Lakhs of Idiots, uneducated Tamil ppl want to lift ban on # jalikattu 2play with animals to hurt them. So how can u expect gud from such ppl? [SEP] don't dare to say this... come to merina and say the same thing. Surely u will come to know who are Tamil people [SEP]


In [ ]:
print(len(train_data[0]["token_type_ids"]))
print(len(train_data[0]["targets"]))
print(len(train_data[0]["ids"]))
print(len(train_data[0]["mask"]))

#print(train_data[0]["text_length"])
#print(train_data[0]["pt_length"])
#print(train_data[0]["topic_length"])

print(train_data[0]["token_type_ids"])
print(train_data[0]["ids"])


510
2
510
510
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 